In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1
import torch

from train import init_model, init_training, create_connections_datasets

env: CUDA_LAUNCH_BLOCKING=1


/home/brianf/minRL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer, model = init_model()

2025-05-09 19:24:54.141 | INFO     | train:init_model:42 - Model loaded.
2025-05-09 19:24:54.146 | INFO     | train:init_model:45 - Using device cuda:0


In [3]:
train_dataset, _ = create_connections_datasets(tokenizer)

In [4]:
training_objects = init_training(model, train_dataset)

In [5]:
from train import init_training
from grpo import rollout, update_policy

from tasks.countdown import reward_function

to = init_training(model, train_dataset)
batch = next(iter(to.train_dataloader))

episodes = rollout(
    model=model,
    tokenizer=tokenizer,
    batch=batch,
    max_new_tokens=to.config.max_new_tokens,
    num_answer_per_question=to.config.num_answer_per_question,
    reward_function=reward_function,
    device=to.device,
)
if to.config.skip_unfinished_episodes:
    episodes = [episode for episode in episodes if episode.is_finished]

2025-05-09 19:25:02.502 | INFO     | grpo:rollout:43 - Generating responses for 2 prompts, max_tokens=256
2025-05-09 19:26:05.523 | INFO     | grpo:rollout:74 - Generated token ids: 256
2025-05-09 19:26:05.524 | INFO     | grpo:rollout:81 - Generated token ids after removing padding: 256
2025-05-09 19:26:05.525 | INFO     | grpo:rollout:88 - Generated text: 

Okay, let's see. The user provided a list of words: bottom, candy, garden, star, boo, honey, sugar, sweetie, arizona, colorado, nevada, utah, comb, hive, honey, wax. They want me to form four groups of four words each, with each word used only once per group. 

First, I need to look for words that can form a group. Let's start with the first group. The words "bottom," "candy," "garden," and "star" all seem to be related to nature. "Bottom" could be part of a place, "candy" and "honey" as sweet things, "garden" and "star" as natural elements. That makes sense. So Group 1: bottom, candy, garden, star.

Next, looking for another grou

rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}
rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}


In [6]:
episodes[0].reward

0.0

In [ ]:
# Update policy - compute loss and perform backward pass
results = update_policy(
    model=model,
    optimizer=to.optimizer,
    episodes=episodes,
    micro_batch_size=to.config.micro_batch_size,
    pad_token_id=tokenizer.pad_token_id,
    max_grad_norm=to.config.max_grad_norm,
    device=to.device,
    dtype=to.dtype,
    apply_loss=True
)


episodes [0.0, 0.0]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
results